In [136]:
from skimage.morphology import medial_axis
import numpy as np
from pymks import PrimitiveTransformer
from sklearn.preprocessing import OneHotEncoder
from toolz.curried import map as fmap
from toolz.curried import curry, pipe

In [141]:
@curry
def one_hot_encode(data, n_state):
    return np.array(PrimitiveTransformer(n_state=n_state, min_=0, max_=n_state - 1).fit_transform(data))

unwrap = lambda x: np.array(list(zip(*x)))

@curry
def moveaxis(ind0, ind1, x):
    return np.moveaxis(x, ind0, ind1)

def skeletonize(data):
    ## this evaluates the skeleton function across multiple samples and two phases
    skeleton_func = fmap(lambda x: medial_axis(x, return_distance=True))
    return pipe(
        data,
        skeleton_func,  
        unwrap,  ## each skeleton and distance map is in a seperate tuple for each sample(combine skeletons and distance maps together)
        tuple
    )


def skeletonize_multiphase(data, n_state):
    return pipe(
        data,
        one_hot_encode(n_state=n_state),
        moveaxis(-1, 0),
        fmap(skeletonize),
        unwrap,
        moveaxis(1, -1),
        tuple
    )
#    _ = one_hot_encode(data, n_phases)
#    _ = np.moveaxis(_, -1, 0)
##    _ = fmap(lambda x: skeletonize(x), _)
#    _ = unwrap(_)
#    _ = np.moveaxis(_, 1, -1)
#    return tuple(_) 


In [142]:
data = np.random.randint(0, 4, size=(5, 3, 3))
#data_encoded = one_hot_encode(data, 3)
#data_encoded.shape

#s, d = skeletonize(data)

s, d = skeletonize_multiphase(data, 4)

#np.moveaxis(np.array(list(fmap(lambda x: skeletonize(x)[0], np.moveaxis(data_encoded, -1, 0)))), 0, -1).shape

In [143]:
print(s.shape)
print(d.shape)

(5, 3, 3, 4)
(5, 3, 3, 4)


In [98]:
np.moveaxis(data_encoded, -1, 0).shape

(3, 4, 5, 5)

In [87]:
s

array([[[ True,  True, False, False, False],
        [False, False,  True, False, False],
        [False, False, False,  True, False],
        [False, False, False, False,  True],
        [False,  True,  True,  True, False]],

       [[False,  True, False,  True, False],
        [ True, False,  True, False,  True],
        [ True, False,  True, False,  True],
        [ True, False, False,  True, False],
        [ True, False, False, False, False]],

       [[False, False, False, False, False],
        [False, False,  True, False, False],
        [False,  True, False,  True, False],
        [False,  True, False,  True, False],
        [False, False,  True, False, False]]])

In [74]:
s.shape

(1, 5, 5)

In [91]:
aa = [('s0', 'd0'), ('s1', 'd1'), ('s2', 'd2'), ('s3', 'd3')]
list(zip(*aa))

[('s0', 's1', 's2', 's3'), ('d0', 'd1', 'd2', 'd3')]

In [89]:
aa = np.arange(9).reshape(3, 3)
bb = np.arange(4).reshape(2, 2)

aa_, bb_ = list(zip(*[(aa, bb), (aa, bb), (aa, bb), (aa, bb)]))
print(aa_)
print(np.stack(aa_))
#np.stack(bb_).shape

(array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]]), array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]]), array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]]), array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]]))
[[[0 1 2]
  [3 4 5]
  [6 7 8]]

 [[0 1 2]
  [3 4 5]
  [6 7 8]]

 [[0 1 2]
  [3 4 5]
  [6 7 8]]

 [[0 1 2]
  [3 4 5]
  [6 7 8]]]


In [58]:
data = np.random.randint(0, 3, size=(1, 5, 5))
print(data)
data_encoded = one_hot_encode(data, 3)
print(data_encoded.shape)





[[[0 1 0 2 0]
  [1 0 2 1 0]
  [0 0 1 1 0]
  [0 0 1 1 1]
  [2 1 0 2 1]]]
(1, 5, 5, 3)


In [30]:
x_data = np.random.randint(0, 3, size=(1, 5, 5))
print(x_data)
x_trans = PrimitiveTransformer(n_state=3, min_=0, max_=2).fit_transform(x_data)
print(np.array(x_trans))

[[[2 0 2 0 1]
  [1 2 0 2 1]
  [0 0 1 1 1]
  [0 2 1 0 0]
  [2 0 2 1 0]]]
[[[[0. 0. 1.]
   [1. 0. 0.]
   [0. 0. 1.]
   [1. 0. 0.]
   [0. 1. 0.]]

  [[0. 1. 0.]
   [0. 0. 1.]
   [1. 0. 0.]
   [0. 0. 1.]
   [0. 1. 0.]]

  [[1. 0. 0.]
   [1. 0. 0.]
   [0. 1. 0.]
   [0. 1. 0.]
   [0. 1. 0.]]

  [[1. 0. 0.]
   [0. 0. 1.]
   [0. 1. 0.]
   [1. 0. 0.]
   [1. 0. 0.]]

  [[0. 0. 1.]
   [1. 0. 0.]
   [0. 0. 1.]
   [0. 1. 0.]
   [1. 0. 0.]]]]


In [47]:
x_data = np.random.randint(0, 2, size=(5, 5)).flatten().reshape(25, 1)
print(x_data.shape)

(25, 1)


In [50]:
ohe = OneHotEncoder(sparse=False, categories=np.array([[0, 1, 2]])).fit(x_data)

/nix/store/qd9v8ihxx0ybq8fj8q4xb2irqzzzrh83-python3.9-scikit-learn-1.0.2/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:82: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.categories != "auto":
/nix/store/qd9v8ihxx0ybq8fj8q4xb2irqzzzrh83-python3.9-scikit-learn-1.0.2/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:93: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.categories == "auto":


In [51]:
ohe.transform(x_data)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [41]:
x_data

array([[0],
       [1],
       [1],
       [2],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [2],
       [0],
       [0],
       [2],
       [2],
       [0],
       [2],
       [0],
       [2],
       [1]])